# Note:  To view project and maps perfectly Just open the url: https://nbviewer.jupyter.org/ and copy the github address of this project and then go! 

<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">ML Capstone Project - 4Square Churches  Part B</font></h1>

Now that the scenario of using existing churches has been solved, let's look at the scenario where a decision needs to be made about where to locate a new church.  In other words, if the decision to locate a new church in Toronto is based solely on where it is easiest to meet, talk, and socialize with people i.e. cafes, coffee houses , which neighborhood should the new church be located.

The method to arrive at this decision is as follows.
1. Apply K-Means cluster analysis to the data in order to find the neighborhood with the attribute of having many cafes / Coffee Houses.  This is the same analysis as done in the assignment for Week 3 of this Capstone Project.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#! pip install folium==0.5.0
import folium # map rendering library

from bs4 import BeautifulSoup


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    



import geocoder
import matplotlib
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#print('Folium installed')
print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
import bs4
print(bs4.__version__)


4.9.3


In [ ]:
#!conda install -c conda-forge geopy --yes
#!conda uninstall -c conda-forge geopy --yes

## 1. Scrape and Create the Dataframe
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 
    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


### To scrape the data from the website, use the library 'requests' to open and return the html.  Then parse it using the beautifulsoup library.  Put it in a pandas dataframe & clean it & define its shape.


In [3]:
# get the information about Post Code of the neighborhoods of Toronto from wikipage and clean it
#Use function “prettify” to look at nested structure of HTML page
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')
#print(soup.prettify())


In [4]:
#Extract the First Table using the python 'find' command; display the table
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [5]:
#Extract the Postal Codes from the HTML table
#First find all the Table Rows i.e. all the rows with data
table_rows = table.find_all('tr')
data = []  #Initialize the list to contain the rows
for row in table_rows:   #Loop through each row in the table rows list
    td=[]  #Initialize the list to contain each individual row
    for t in row.find_all('td'):  #Loop through each element in the list of data in a given row
        td.append(t.text.strip())  #append each element in the list: td.
    data.append(td) #gather each list 'td' into the 'data' list
    
#the pd.set_option simply allows us to see all the data in the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
neighborhoods = pd.DataFrame(data, columns=['Postal_Code', 'Borough', 'Neighborhood'])  #Convert the list data into a Panda Dataframe

In [6]:
neighborhoods.drop(neighborhoods[neighborhoods.Borough == 'Not assigned'].index, inplace=True)
neighborhoods.reset_index(drop=True, inplace=True)

#Below, the groupby is applied to all three columns.  So effectively, each column is deduped within itself and 
#then the last column (Neighborhood), the unique values are joined / concatenated with ',' and the unique values.
neighborhoods = neighborhoods.groupby(['Postal_Code','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
neighborhoods





,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
neighborhoods.shape

(103, 3)

### Get the Latitude and Longitude of the Coordinates for each Postal Code - Merge them to the Pandas DF created above

### geocoder Failed to pull in any data so went to .csv

In [8]:
latlong_neighborhoods = pd.read_csv("C:\\Users\\StephenVoorhees\\Documents\\_Training\\Python\\Projects\\ML-Capstone2\\Battle_of_Neighborhoods\\Geospatial_Coordinates.csv") 
neighborhoods= pd.merge(neighborhoods,latlong_neighborhoods, how= 'inner', on='Postal_Code')
neighborhoods.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [10]:
address = '33 Wanless Crescent, Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.72681267221106, -79.3912508694319.


### Create a Map of Toronto (as in the NYC example) with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's look at Neighborhoods across Boroughs i.e. for all of Toronto

In [15]:
neighborhoods_data = neighborhoods
neighborhoods_data

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Let's get the geographical coordinates of Toronto.


Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = '4NHYUOY1R4VMOQ04MJW2FC2AUOVCYTVLKU53CHBLMXBI5RTU' # your Foursquare ID
CLIENT_SECRET = 'CLPUAU4CNGQ45HH01RM24CN2YXVI4PYSDBSPNEZ0R3E5XIZJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4NHYUOY1R4VMOQ04MJW2FC2AUOVCYTVLKU53CHBLMXBI5RTU
CLIENT_SECRET:CLPUAU4CNGQ45HH01RM24CN2YXVI4PYSDBSPNEZ0R3E5XIZJ


#### Explore Toronto's the first neighborhood in our dataframe by getting the neighborhood's name, Latitude, Longitude

#### Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 500 meters.  First, let's create the GET request URL. Name your URL: url.

In [19]:
neighborhood_latitude = neighborhoods_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # 1609 Meters in a mile - define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=4NHYUOY1R4VMOQ04MJW2FC2AUOVCYTVLKU53CHBLMXBI5RTU&client_secret=CLPUAU4CNGQ45HH01RM24CN2YXVI4PYSDBSPNEZ0R3E5XIZJ&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc6a673a1e37a62bad123ae'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

## 2. Explore Neighborhoods in Toronto
#### create a function to repeat the same process to all the neighborhoods inToronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods_data['Neighborhood'],
                                   latitudes=neighborhoods_data['Latitude'],
                                   longitudes=neighborhoods_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [26]:
print(neighborhoods_venues.shape)
neighborhoods_venues.head(10)

(2141, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and Building Solutions),43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


Let's check how many venues were returned for each neighborhood


In [28]:
neighborhoods_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22


#### Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} unique categories of venues.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 273 unique categories of venues.


In [32]:
#create a list of uniques categories
List = neighborhoods_venues.groupby('Venue Category')['Venue'].count().sort_values(ascending=False)
List

Venue Category
Coffee Shop                        192
Café                                93
Restaurant                          67
Park                                51
Pizza Place                         50
Hotel                               44
Sandwich Place                      43
Bakery                              41
Japanese Restaurant                 41
Italian Restaurant                  40
Gym                                 34
Bar                                 34
Clothing Store                      33
Sushi Restaurant                    29
Grocery Store                       26
Pub                                 25
American Restaurant                 25
Fast Food Restaurant                24
Bank                                24
Breakfast Spot                      23
Thai Restaurant                     22
Seafood Restaurant                  22
Pharmacy                            21
Beer Bar                            19
Gastropub                           19
Bookstore 

### 3. Analyze Each Neighborhood

In [33]:
# one hot encoding i.e. create a file with dummy values to indicate that that venue was present in the neighborhood
neighborhoods_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
fixed_columns = [neighborhoods_onehot.columns[191]] + list(neighborhoods_onehot.columns[0:190]) + list(neighborhoods_onehot.columns[192:-1])  + [neighborhoods_onehot.columns[-1]]
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]

neighborhoods_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [34]:
#New DataFrame size:
neighborhoods_onehot.shape

(2141, 272)

#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [80]:
#Note the sum() of the frequency of occurence of each venue category was used instead of the mean().  The sum() gave
#higher Sillouette scores where more meaningful distinctions were make between clusters.  i.e. if 5 clusters
#were chosen for kMeans, then 4 of the 5 clusters had most common venues that did not have much differences.  However,
#if two clusters were used, the Venue categories were, in fact, different/distinct from the two clusters.  That is,
#The distinction between Downtown Toronto Neighborhoods and Neighborhoods outside of downtown were pronounced: Downtown 
#had mostly Coffee shops as the 1st most common venue category and outside downtown did not have coffee shops as 
#the 1st most common venue category (in general).
neighborhoods_grouped = neighborhoods_onehot.groupby('Neighborhood').sum().reset_index()
neighborhoods_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [81]:
neighborhoods_grouped.shape

(96, 272)

#### Print each neighborhood along with the top 5 most common venues

In [82]:
num_top_venues = 10

for hood in neighborhoods_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhoods_grouped[neighborhoods_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                   Breakfast Spot   1.0
1                           Lounge   1.0
2                     Skating Rink   1.0
3        Latin American Restaurant   1.0
4                   Clothing Store   1.0
5                Accessories Store   0.0
6        Middle Eastern Restaurant   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9                Mobile Phone Shop   0.0


----Alderwood, Long Branch----
                  venue  freq
0           Pizza Place   2.0
1                   Gym   1.0
2        Sandwich Place   1.0
3                   Pub   1.0
4           Coffee Shop   1.0
5              Pharmacy   1.0
6  Other Great Outdoors   0.0
7             Pet Store   0.0
8   Martial Arts School   0.0
9        Massage Studio   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop   2.0
1                       Bank   2.0
2    

9   Middle Eastern Restaurant   0.0


----India Bazaar, The Beaches West----
                  venue  freq
0                  Park   2.0
1           Pizza Place   1.0
2        Sandwich Place   1.0
3      Sushi Restaurant   1.0
4  Fast Food Restaurant   1.0
5            Restaurant   1.0
6         Movie Theater   1.0
7    Italian Restaurant   1.0
8          Liquor Store   1.0
9           Coffee Shop   1.0


----Kennedy Park, Ionview, East Birchmount Park----
                             venue  freq
0                      Coffee Shop   1.0
1                    Train Station   1.0
2                       Hobby Shop   1.0
3                 Department Store   1.0
4              Monument / Landmark   0.0
5  Molecular Gastronomy Restaurant   0.0
6       Modern European Restaurant   0.0
7                Mobile Phone Shop   0.0
8               Miscellaneous Shop   0.0
9        Middle Eastern Restaurant   0.0


----Kensington Market, Chinatown, Grange Park----
                           venue  fr

9  Middle Eastern Restaurant   1.0


----Toronto Dominion Centre, Design Exchange----
                 venue  freq
0          Coffee Shop  14.0
1                Hotel   8.0
2           Restaurant   5.0
3                 Café   5.0
4          Salad Place   3.0
5  American Restaurant   3.0
6   Seafood Restaurant   3.0
7  Japanese Restaurant   3.0
8  Sporting Goods Shop   2.0
9     Asian Restaurant   2.0


----University of Toronto, Harbord----
                 venue  freq
0                 Café   5.0
1            Bookstore   3.0
2                  Bar   2.0
3  Japanese Restaurant   2.0
4       Sandwich Place   2.0
5               Bakery   2.0
6          Yoga Studio   1.0
7          Coffee Shop   1.0
8     Sushi Restaurant   1.0
9             Beer Bar   1.0


----Victoria Village----
                             venue  freq
0                      Pizza Place   1.0
1                     Hockey Arena   1.0
2            Portuguese Restaurant   1.0
3                     Intersection   1.0
4  

#### Let's put that into a _pandas_ dataframe
First, let's write a function to sort the venues in descending order.

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(103)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,Gourmet Shop,Dance Studio,Dumpling Restaurant,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Mobile Phone Shop,Diner,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Fried Chicken Joint,Ice Cream Shop
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Café,Restaurant
5,Berczy Park,Coffee Shop,Restaurant,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Sandwich Place,Breakfast Spot
6,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Stadium,Restaurant,Italian Restaurant,Bar
8,"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",Skate Park,Farmers Market,Restaurant,Butcher,Recording Studio,Brewery,Auto Workshop,Garden,Garden Center,Burrito Place
9,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Airport Lounge,Airport Service,Boutique,Sculpture Garden,Rental Car Location,Coffee Shop,Plane,Harbor / Marina,Boat or Ferry,Airport Terminal


## 4. Cluster Neighborhoods
Run _k_-means to cluster the neighborhood into 5 clusters.


In [86]:
# determine the optimal value of K for our dataset using the Silhouette Coefficient Method
neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(neighborhoods_grouped_clustering)
    label = kmeans.labels_
    sil_coeff = silhouette_score(neighborhoods_grouped_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.5983807500173547
For n_clusters=3, The Silhouette Coefficient is 0.3539252481718003
For n_clusters=4, The Silhouette Coefficient is 0.42319689408339856
For n_clusters=5, The Silhouette Coefficient is 0.43756168340218077
For n_clusters=6, The Silhouette Coefficient is 0.36702871693252986
For n_clusters=7, The Silhouette Coefficient is 0.4193412600887358
For n_clusters=8, The Silhouette Coefficient is 0.34507947647748155
For n_clusters=9, The Silhouette Coefficient is 0.3229878822887474


In [87]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 2

neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [88]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_merged = neighborhoods_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_merged = neighborhoods_merged.dropna()
neighborhoods_merged # check the last columns!

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Restaurant,Electronics Store,Rental Car Location,Medical Center,Bank,Mexican Restaurant,Dog Run,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Caribbean Restaurant,Hakka Restaurant,Bakery,Gas Station,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Distribution Center
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Train Station,Hobby Shop,Coffee Shop,Department Store,Escape Room,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Dim Sum Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0.0,Bus Line,Bakery,Park,Ice Cream Shop,Bus Station,Metro Station,Intersection,Soccer Field,Creperie,Dog Run
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant


In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighborhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish 
each cluster. Based on the defining categories, you can then assign a name to each cluster. 


In [90]:
neighborhoods_merged

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Restaurant,Electronics Store,Rental Car Location,Medical Center,Bank,Mexican Restaurant,Dog Run,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Caribbean Restaurant,Hakka Restaurant,Bakery,Gas Station,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Distribution Center
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Train Station,Hobby Shop,Coffee Shop,Department Store,Escape Room,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Dim Sum Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0.0,Bus Line,Bakery,Park,Ice Cream Shop,Bus Station,Metro Station,Intersection,Soccer Field,Creperie,Dog Run
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant


In [91]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,Scarborough,0.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,0.0,Intersection,Breakfast Spot,Restaurant,Electronics Store,Rental Car Location,Medical Center,Bank,Mexican Restaurant,Dog Run,Discount Store
3,Scarborough,0.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,Scarborough,0.0,Caribbean Restaurant,Hakka Restaurant,Bakery,Gas Station,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Distribution Center
5,Scarborough,0.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,0.0,Train Station,Hobby Shop,Coffee Shop,Department Store,Escape Room,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Dim Sum Restaurant
7,Scarborough,0.0,Bus Line,Bakery,Park,Ice Cream Shop,Bus Station,Metro Station,Intersection,Soccer Field,Creperie,Dog Run
8,Scarborough,0.0,American Restaurant,Motel,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant


In [92]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 1, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
53,Downtown Toronto,1.0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Café,Theater,Distribution Center,Electronics Store,Bank
54,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Pizza Place,Hotel,Furniture / Home Store,Bookstore
55,Downtown Toronto,1.0,Coffee Shop,Café,Restaurant,Cocktail Bar,American Restaurant,Beer Bar,Gastropub,Seafood Restaurant,Hotel,Japanese Restaurant
57,Downtown Toronto,1.0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Thai Restaurant,Salad Place,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Department Store
58,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Gym,Restaurant,Bar,Clothing Store,Thai Restaurant,Bookstore,Office
59,Downtown Toronto,1.0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Pizza Place,Fried Chicken Joint,Brewery,Restaurant,Italian Restaurant
60,Downtown Toronto,1.0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Asian Restaurant,Deli / Bodega
61,Downtown Toronto,1.0,Coffee Shop,Restaurant,Café,Hotel,Gym,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,American Restaurant
69,Downtown Toronto,1.0,Coffee Shop,Italian Restaurant,Restaurant,Seafood Restaurant,Pub,Café,Hotel,Beer Bar,Japanese Restaurant,Breakfast Spot
